In [2]:
# Install required dependencies and import necessary libraries
!pip install codecarbon #profiler
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import os
import time
from tqdm import tqdm
from codecarbon import EmissionsTracker

# Set device (Use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
EPOCHS = 20
BATCH_SIZE = 64  # Reduce batch size to avoid memory issues
LR = 0.01
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9
NUM_CLASSES = 100  # CIFAR-100



In [3]:
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

# Correct order: Apply AutoAugment before ToTensor()
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    AutoAugment(AutoAugmentPolicy.CIFAR10),  # Must be applied before ToTensor()
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    transforms.RandomErasing(p=0.2)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))
])


In [4]:
# Load CIFAR-100 dataset and prepare dataloaders

trainset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


100%|██████████| 169M/169M [00:02<00:00, 69.2MB/s] 


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Define ResNet model with Residual Blocks
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return torch.relu(out)

class ResNet(nn.Module):
    def __init__(self, block, num_classes=100):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, 2, stride=1)
        self.layer2 = self._make_layer(block, 128, 2, stride=2)
        self.layer3 = self._make_layer(block, 256, 2, stride=2)
        self.layer4 = self._make_layer(block, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        for _ in range(num_blocks):
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.flatten(out, start_dim=1)
        return self.fc(out)

# Initialize Model
model = ResNet(ResidualBlock, num_classes=NUM_CLASSES).to(device)


In [6]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label smoothing for better generalization
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)


In [ ]:
tracker = EmissionsTracker(log_level="critical", output_file="emissions.csv", output_dir="./")

def train_and_evaluate():
    best_acc = 0.0
    total_start_time = time.time()
    tracker.start()

    for epoch in range(EPOCHS):
        epoch_start_time = time.time()
        model.train()
        correct, total, running_loss = 0, 0, 0.0

        progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch")
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{100. * correct / total:.2f}%")

        train_acc = 100. * correct / total
        scheduler.step()
        epoch_time = time.time() - epoch_start_time
        print(f"Epoch {epoch+1}: Train Acc: {train_acc:.2f}%, Time: {epoch_time:.2f}s")

    tracker.stop() # Stop emissions tracking and display total CO₂ emissions
    total_time = time.time() - total_start_time
    print(f"Total Training Time: {total_time:.2f}s")

train_and_evaluate()
